In [1]:
import pandas as pd
import numpy as np
import os
import pandas_profiling

In [2]:
file_path = os.path.join("data", "twitter_URLs.bin")
df = pd.read_csv(file_path)
df.head()

,expanded_url,domain,tag
0,https://twitter.com/kjh_mov/status/12996294579...,twitter.com,NaN
1,https://twitter.com/kjh_mov/status/12996294579...,twitter.com,NaN
2,https://twitter.com/kjh_mov/status/12996294579...,twitter.com,NaN
3,http://ntv.de,ntv.de,NaN
4,https://twitter.com/kjh_mov/status/12996294506...,twitter.com,NaN


In [3]:
df.describe()

,expanded_url,domain,tag
count,401411,401411,27904
unique,286502,9948,9
top,https://support.twitter.com/articles/20169199,twitter.com,Youtube
freq,623,288363,25311


In [4]:
df["domain"].value_counts()

twitter.com          288363
youtu.be              14964
youtube.com           10339
welt.de                1836
tagesspiegel.de        1804
                      ...  
vital.de                  1
home.mycloud.com          1
gerecht.er                1
andrewhammel.blog         1
karrenbrock.at            1
Name: domain, Length: 9948, dtype: int64

In [5]:
df = df.loc[df.domain == "bild.de"]
df.describe()

,expanded_url,domain,tag
count,1153,1153,0
unique,558,1,0
top,https://www.bild.de/politik/inland/politik-inl...,bild.de,NaN
freq,85,1153,NaN


In [6]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.describe()

,expanded_url,domain,tag
count,558,558,0
unique,558,1,0
top,https://m.bild.de/ratgeber/2021/ratgeber/studi...,bild.de,NaN
freq,1,558,NaN


### Newspaper3k

In [7]:
from newspaper import Article

#create new columns for article title, text, summary and keywords
df["title"] = np.nan
df["text"] = np.nan
df["summary"] = np.nan
df["keywords"] = np.nan

#extract title, text, summary and keywords for each article and copy to corresponding row-column
url_list = df.expanded_url
i_max = 20
for i in range(0,i_max):
    print("Article number: "+str(i))
    try:
        article = Article(url_list[i], language="de")
        article.download()
        article.parse()
        article.nlp()
    
        df["title"][i] = article.title
        df["text"][i] = article.text
        df["summary"][i] = article.summary
        df["keywords"][i] = article.keywords
    except Exception as e:
        print(e)

Article number: 0


<ipython-input-7-2b57538d9adc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"][i] = article.title
C:\Users\Shabb\anaconda3\envs\ML\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-7-2b57538d9adc>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i] = article.text
<ipython-input-7-2b57538d9adc>:22: Settin

Article number: 1
Article number: 2
Article number: 3
Article number: 4
Article number: 5
Article number: 6
Article number: 7
Article number: 8
Article number: 9
Article number: 10
Article number: 11
Article number: 12
Article number: 13
Article number: 14
Article number: 15
Article number: 16
Article number: 17
Article number: 18
Article number: 19


In [8]:
#remove all rows which has no keywords and reset index
df = df[~df['keywords'].isnull()]
df = df.reset_index(drop=True)
df.describe()

,expanded_url,domain,tag,title,text,summary,keywords
count,20,20,0,20,20,20,20
unique,20,1,0,13,15,15,14
top,https://m.bild.de/politik/ausland/politik-ausl...,bild.de,NaN,Kaum noch Bedarf Uniklinik Leipzig schließt Co...,Leipzig – Die Corona-Ambulanz des Universitäts...,Leipzig – Die Corona-Ambulanz des Universitäts...,"[leipzig, bedarf, flughafen, leipzighalle, sch..."
freq,1,20,NaN,6,6,6,6


In [9]:
df.isna().sum()

expanded_url     0
domain           0
tag             20
title            0
text             0
summary          0
keywords         0
dtype: int64

In [10]:
df

,expanded_url,domain,tag,title,text,summary,keywords
0,https://m.bild.de/regional/berlin/berlin-aktue...,bild.de,NaN,Linken-Demo gegen Räumung von Berliner Kneipe ...,Berlin – Die Hauptstadt im Protest-Modus. Nich...,Nicht nur die Corona-Leugner sorgen am Samstag...,"[zwangsräumung, zeigenam, via, vormülleimer, l..."
1,https://m.bild.de/politik/ausland/politik-ausl...,bild.de,NaN,Neuer Trump-Fauxpas - „Pandemie beendete wohl ...,veröffentlicht am\n\nDonald Trump (74) und die...,veröffentlicht amDonald Trump (74) und die Wel...,"[neuer, netzwerken, sozialen, pandemie, weltkr..."
2,https://www.bild.de/regional/leipzig/leipzig-n...,bild.de,NaN,Kaum noch Bedarf Uniklinik Leipzig schließt Co...,Leipzig – Die Corona-Ambulanz des Universitäts...,Leipzig – Die Corona-Ambulanz des Universitäts...,"[leipzig, bedarf, flughafen, leipzighalle, sch..."
3,https://m.bild.de/regional/leipzig/leipzig-new...,bild.de,NaN,Kaum noch Bedarf Uniklinik Leipzig schließt Co...,Leipzig – Die Corona-Ambulanz des Universitäts...,Leipzig – Die Corona-Ambulanz des Universitäts...,"[leipzig, bedarf, flughafen, leipzighalle, sch..."
4,https://www.bild.de/politik/inland/politik-inl...,bild.de,NaN,Anti-Israel-Demo in Berlin: Hass auf Juden!,Es sollte ein Hetz-Auftritt gegen Juden werden...,Im Herzen der Hauptstadt vor dem Brandenburger...,"[kundgebung, berlin, brandenburger, kampf, avi..."
5,https://www.bild.de/regional/muenchen/muenchen...,bild.de,NaN,Wegen Unverhältnismäßigkeit: Gericht kippt Alk...,Von: JAKOB MELL und OLIVER GROTHMANN\n\nMünche...,Das Verwaltungsgericht München hat das Alkohol...,"[tage, kippt, uhr, gericht, verwaltungsgericht..."
6,https://www.bild.de/regional/hannover/hannover...,bild.de,NaN,Corona-Urteil in Niedersachsen: Gericht kippt ...,"Bordelle, Straßenprostitution, Lovemobile – in...",Das niedersächsische Oberverwaltungsgericht ha...,"[möglichen, niedersachsen, oberverwaltungsgeri..."
7,https://www.bild.de/regional/berlin/berlin-akt...,bild.de,NaN,Corona-Demo in Berlin kann stattfinden - 3000 ...,Die Entscheidung des Berliner Verwaltungsgeric...,Berlin – Die Berliner Polizei akzeptiert die E...,"[3000, berlin, demonstration, polizisten, coro..."
8,https://m.bild.de/regional/berlin/berlin-aktue...,bild.de,NaN,Corona-Demo in Berlin kann stattfinden - 3000 ...,veröffentlicht am\n\nDie Entscheidung des Berl...,Berlin – Die Berliner Polizei akzeptiert die E...,"[3000, berlin, demonstration, polizisten, coro..."
9,https://m.bild.de/politik/2020/politik/berline...,bild.de,NaN,Berliner Verwaltungsgericht: Verbot von Demos ...,veröffentlicht am\n\nDas Berliner Verwaltungsg...,veröffentlicht amDas Berliner Verwaltungsgeric...,"[berlin, ziehen, coronademos, coronapolitik, a..."


### Gensim

In [11]:
import gensim

def gensim_summarization(corpus, ratio=0.2):
    if type(corpus) is str:        
       corpus = [corpus]    
    summary = [gensim.summarization.summarize(text, ratio=ratio) for text in corpus]    
    return summary

In [12]:
df["gen_summ_10"] = np.nan
df["gen_summ_15"] = np.nan
df["gen_summ_20"] = np.nan

for i in range(20):
    text_summ_i = gensim_summarization(df["text"][i], ratio=0.1)
    df["gen_summ_10"][i] = text_summ_i

for i in range(20):
    text_summ_i = gensim_summarization(df["text"][i], ratio=0.15)
    df["gen_summ_15"][i] = text_summ_i

for i in range(20):
    text_summ_i = gensim_summarization(df["text"][i])
    df["gen_summ_20"][i] = text_summ_i

<ipython-input-12-590d7b70d10f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["gen_summ_10"][i] = text_summ_i
C:\Users\Shabb\anaconda3\envs\ML\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-12-590d7b70d10f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["gen_summ_15"][i] = text_summ_i
<ipython-input-12-590d7b70d10f

In [13]:
from gensim.summarization import keywords

df["gen_keywords"] = ""

for i in range(20):
    keywords_i = keywords(df["text"][i], ratio=0.2, split=",")
    df["gen_keywords"][i] = keywords_i

In [14]:
df

,expanded_url,domain,tag,title,text,summary,keywords,gen_summ_10,gen_summ_15,gen_summ_20,gen_keywords
0,https://m.bild.de/regional/berlin/berlin-aktue...,bild.de,NaN,Linken-Demo gegen Räumung von Berliner Kneipe ...,Berlin – Die Hauptstadt im Protest-Modus. Nich...,Nicht nur die Corona-Leugner sorgen am Samstag...,"[zwangsräumung, zeigenam, via, vormülleimer, l...",[Nach BILD-Informationen waren circa 2000 Demo...,[Nach BILD-Informationen waren circa 2000 Demo...,[Nicht nur die Corona-Leugner sorgen am Samsta...,"[die, der, und, wurden nach, linken, auf, ecke..."
1,https://m.bild.de/politik/ausland/politik-ausl...,bild.de,NaN,Neuer Trump-Fauxpas - „Pandemie beendete wohl ...,veröffentlicht am\n\nDonald Trump (74) und die...,veröffentlicht amDonald Trump (74) und die Wel...,"[neuer, netzwerken, sozialen, pandemie, weltkr...","[Trump versprach sich vermutlich, als er am Mo...","[Trump versprach sich vermutlich, als er am Mo...","[Trump versprach sich vermutlich, als er am Mo...","[und die, den, weltkrieg, weltkrieges, einer, ..."
2,https://www.bild.de/regional/leipzig/leipzig-n...,bild.de,NaN,Kaum noch Bedarf Uniklinik Leipzig schließt Co...,Leipzig – Die Corona-Ambulanz des Universitäts...,Leipzig – Die Corona-Ambulanz des Universitäts...,"[leipzig, bedarf, flughafen, leipzighalle, sch...",[],[Leipzig – Die Corona-Ambulanz des Universität...,[Leipzig – Die Corona-Ambulanz des Universität...,"[die, auf, der, leipzig, sich, mit, lassen, co..."
3,https://m.bild.de/regional/leipzig/leipzig-new...,bild.de,NaN,Kaum noch Bedarf Uniklinik Leipzig schließt Co...,Leipzig – Die Corona-Ambulanz des Universitäts...,Leipzig – Die Corona-Ambulanz des Universitäts...,"[leipzig, bedarf, flughafen, leipzighalle, sch...",[],[Leipzig – Die Corona-Ambulanz des Universität...,[Leipzig – Die Corona-Ambulanz des Universität...,"[die, auf, der, leipzig, sich, mit, lassen, co..."
4,https://www.bild.de/politik/inland/politik-inl...,bild.de,NaN,Anti-Israel-Demo in Berlin: Hass auf Juden!,Es sollte ein Hetz-Auftritt gegen Juden werden...,Im Herzen der Hauptstadt vor dem Brandenburger...,"[kundgebung, berlin, brandenburger, kampf, avi...",[Im Herzen der Hauptstadt vor dem Brandenburge...,[Im Herzen der Hauptstadt vor dem Brandenburge...,[Im Herzen der Hauptstadt vor dem Brandenburge...,"[der, und, doch die, ein, eine, berliner, berl..."
5,https://www.bild.de/regional/muenchen/muenchen...,bild.de,NaN,Wegen Unverhältnismäßigkeit: Gericht kippt Alk...,Von: JAKOB MELL und OLIVER GROTHMANN\n\nMünche...,Das Verwaltungsgericht München hat das Alkohol...,"[tage, kippt, uhr, gericht, verwaltungsgericht...",[Das Verwaltungsgericht hatte am Nachmittag en...,[Das Verwaltungsgericht hatte am Nachmittag en...,[Das Verwaltungsgericht hatte am Nachmittag en...,"[das, die, und, der, auf, fur den, munchen, ei..."
6,https://www.bild.de/regional/hannover/hannover...,bild.de,NaN,Corona-Urteil in Niedersachsen: Gericht kippt ...,"Bordelle, Straßenprostitution, Lovemobile – in...",Das niedersächsische Oberverwaltungsgericht ha...,"[möglichen, niedersachsen, oberverwaltungsgeri...",[Das niedersächsische Oberverwaltungsgericht h...,[Das niedersächsische Oberverwaltungsgericht h...,[Das niedersächsische Oberverwaltungsgericht h...,"[eine, ein, eines, und, oberverwaltungsgericht..."
7,https://www.bild.de/regional/berlin/berlin-akt...,bild.de,NaN,Corona-Demo in Berlin kann stattfinden - 3000 ...,Die Entscheidung des Berliner Verwaltungsgeric...,Berlin – Die Berliner Polizei akzeptiert die E...,"[3000, berlin, demonstration, polizisten, coro...",[Die Entscheidung des Berliner Verwaltungsgeri...,[Die Entscheidung des Berliner Verwaltungsgeri...,[Die Entscheidung des Berliner Verwaltungsgeri...,"[die, eine, und, nicht, nichts, verbot der, da..."
8,https://m.bild.de/regional/berlin/berlin-aktue...,bild.de,NaN,Corona-Demo in Berlin kann stattfinden - 3000 ...,veröffentlicht am\n\nDie Entscheidung des Berl...,Berlin – Die Berliner Polizei akzeptiert die E...,"[3000, berlin, demonstration, p

In [18]:
for i in range(20):
    print("\n*********** article_keywords for index: "+str(i))
    print(df["keywords"][i])
    print("*********** gen_keywords for index: "+str(i))
    print(df["gen_keywords"][i])


*********** article_keywords for index: 0
['zwangsräumung', 'zeigenam', 'via', 'vormülleimer', 'linkendemo', 'vermummte', 'kneipe', 'abend', 'uhr', 'pyros', 'zünden', 'polizei', 'berliner', 'demonstranten', 'räumung', 'wollten', 'weisestraße']
*********** gen_keywords for index: 0
['die', 'der', 'und', 'wurden nach', 'linken', 'auf', 'ecke', 'setzte', 'mit', 'demonstranten vor', 'polizei', 'gegen', 'nicht', 'nebel', 'fur', 'den', 'straßen']

*********** article_keywords for index: 1
['neuer', 'netzwerken', 'sozialen', 'pandemie', 'weltkrieg', 'zustimmung', 'falsch', 'beendete', 'soldaten', 'twitter', 'trumpfauxpas', '1918', 'weltkrieges']
*********** gen_keywords for index: 1
['und die', 'den', 'weltkrieg', 'weltkrieges', 'einer', 'sie', 'trump', 'trumps ist', 'der', 'oder', 'pandemie auf', 'soldaten', 'lange', 'stelle', 'fur sich', 'bei', 'auch', 'ein', 'eine', 'nicht', 'aus']

*********** article_keywords for index: 2
['leipzig', 'bedarf', 'flughafen', 'leipzighalle', 'schließt', '1

In [19]:
for i in range(20):
    print("\n *********** article_summary for index: "+str(i))
    print(df["summary"][i])
    print("*********** gen_summ_10% for index: "+str(i))
    print(df["gen_summ_10"][i])
    print("*********** gen_summ_15% for index: "+str(i))
    print(df["gen_summ_15"][i])
    print("*********** gen_summ_20% for index: "+str(i))
    print(df["gen_summ_20"][i])


 *********** article_summary for index: 0
Nicht nur die Corona-Leugner sorgen am Samstag für einen Großeinsatz der Polizei, sondern auch eine Linken-Demo im Bezirk Neukölln am Abend.
Nachdem sich der Protestzug in Bewegung setzte, kam es Hermannstaße Ecke Flughafenstraße zu Steinwürfen und dem Einsatz von Pyros.
Um 20 Uhr startete die Kundgebung „Lange Nacht der Weisestraße“ gegen die Zwangsräumung der Kiezkneipe Syndikat.
Zudem wollten die Demonstranten Solidarität mit anderen linken Projekten zeigen.
Am Abend beendete der Veranstalter die Demo, da er nicht ausreichend auf die Teilnehmenden einwirken konnte, wie die Polizei via Twitter mitteilte.
*********** gen_summ_10% for index: 0
['Nach BILD-Informationen waren circa 2000 Demonstranten vor Ort. Nachdem sich der Protestzug in Bewegung setzte, kam es Hermannstaße Ecke Flughafenstraße zu Steinwürfen und dem Einsatz von Pyros.']
*********** gen_summ_15% for index: 0
['Nach BILD-Informationen waren circa 2000 Demonstranten vor Ort. Na